## Basic Combustion Analysis
Using Cantera to calculate combustion mechanics.
The one free variable in our design is chamber pressure, for now I will set that at 10 bars
We will be assuming stoichiometric toluene air combustion

In [11]:
import cantera as ct
import numpy as np

# ----- Parameters -----
tank_pressure = 10 * ct.one_atm  # 10 bar
A_throat = 1e-3 * 5e-3 # A 1 x 5 mm slit nozzle

# ----- Use Cantera's built-in thermo with custom composition -----
# We'll compute enthalpy properly

gas = ct.Solution('gri30.yaml')

# Stoichiometric products (before equilibration)
# C7H8 + 9 O2 + 33.84 N2 -> 7 CO2 + 4 H2O + 33.84 N2
stoich_products = 'CO2:7, H2O:4, N2:33.84'

# Enthalpy accounting:
# h_f(C7H8, liquid) = +12 kJ/mol, gas ≈ +50 kJ/mol
# h_f(CO2) = -393.5 kJ/mol
# h_f(H2O, gas) = -241.8 kJ/mol
# Delta_H_combustion = 7*(-393.5) + 4*(-241.8) - 50 = -3672 kJ/mol toluene

# Heat release per kg of stoich mixture:
MW_mix = (92.14 + 9*32 + 33.84*28) / 1  # g per mol toluene worth of mix
heat_release = 3672e3 / (MW_mix / 1000)  # J/kg of mixture

# Air fuel mass weight ratio
m_air = 9*32 + 33.84*28  # g
m_fuel = 92.14  # g
air_fuel_ratio = m_air / m_fuel

# Set products at room temp, then add combustion enthalpy
gas.TPX = 300, tank_pressure, stoich_products
h_products_cold = gas.enthalpy_mass
h_products_hot = h_products_cold + heat_release

# Equilibrate at constant H, P
gas.HP = h_products_hot, tank_pressure
gas.equilibrate('HP')

T_c = gas.T
gamma = gas.cp / gas.cv
MW = gas.mean_molecular_weight
R_sp = 8314.46 / MW

print("="*50)
print("CHAMBER CONDITIONS (Stoichiometric Toluene-Air)")
print("="*50)
print(f"Chamber pressure:    {tank_pressure/1e5:.1f} bar")
print(f"Chamber temperature: {T_c:.0f} K")
print(f"Gamma:               {gamma:.3f}")
print(f"Mean MW:             {MW:.1f} g/mol")
print()

# Choked flow
mdot = (tank_pressure * A_throat * np.sqrt(gamma / (R_sp * T_c)) 
        * (2/(gamma+1))**((gamma+1)/(2*(gamma-1))))

mdot_air = mdot * (air_fuel_ratio / (1 + air_fuel_ratio))
mdot_fuel = mdot - mdot_air


print(f"Throat area:         {A_throat*1e6:.2f} cm²")
print(f"Mass flow rate:      {mdot*1000:.2f} g/s")
print(f"Air mass flow rate:  {mdot_air*1000:.2f} g/s")
print(f"Fuel mass flow rate: {mdot_fuel*1000:.2f} g/s")
print()

print("Major species (>1%):")
for sp, X in zip(gas.species_names, gas.X):
    if X > 0.01:
        print(f"  {sp:8s}: {X*100:.1f}%")

CHAMBER CONDITIONS (Stoichiometric Toluene-Air)
Chamber pressure:    10.1 bar
Chamber temperature: 2356 K
Gamma:               1.250
Mean MW:             29.4 g/mol

Throat area:         5.00 cm²
Mass flow rate:      4.09 g/s
Air mass flow rate:  3.80 g/s
Fuel mass flow rate: 0.28 g/s

Major species (>1%):
  H2O     : 8.6%
  CO      : 1.1%
  CO2     : 14.4%
  N2      : 74.8%


In [14]:
## Calculate flow rate in terms of volumetric flow at STP
R_universal = 8314.46  # J/kmol-K
T_stp = 273.15  # K
P_stp = ct.one_atm  # Pa
Vdot_stp = (mdot / gas.mean_molecular_weight) * R_universal * T_stp / P_stp  # m³/s
Vdot_imperial = Vdot_stp * 1e6 / 1728  # ft³/s
print(f"Volumetric flow rate at STP: {Vdot_stp:.6f} m³/s")
print(f"Volumetric flow rate at STP: {Vdot_imperial:.3f} ft³/s")

Volumetric flow rate at STP: 0.003113 m³/s
Volumetric flow rate at STP: 1.801 ft³/s
